In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import rich

Note: we need to use models that support reasoning like o1 and o3-mini

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "o3-mini"

openai = OpenAI()

Advanced reasoning and problem-solving models

# Chat Completion API

https://platform.openai.com/docs/guides/reasoning?api-mode=chat

In [3]:
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.chat.completions.create(
    model=MODEL,
    reasoning_effort="medium",
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

print(rich.print(response.usage))
print(response.choices[0].message.content)

CompletionUsage(
    completion_tokens=2336,
    prompt_tokens=44,
    total_tokens=2380,
    completion_tokens_details=CompletionTokensDetails(
        accepted_prediction_tokens=0,
        audio_tokens=0,
        reasoning_tokens=1600,
        rejected_prediction_tokens=0
    ),
    prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
)

None
#!/bin/bash
# This script takes a matrix string in the form:
#   "[1,2],[3,4],[5,6]"
# and prints its transpose in the same format.
#
# For example, given:
#   "[1,2],[3,4],[5,6]"
# it outputs:
#   "[1,3,5],[2,4,6]"

# Check if an argument is supplied.
if [ -z "$1" ]; then
  echo "Usage: $0 '[matrix_string]'" >&2
  exit 1
fi

input="$1"

# Remove leading '[' and trailing ']' from the overall string.
# The input format is assumed to be like: "[1,2],[3,4],[5,6]"
input="${input#\[}"
input="${input%\]}"

# Now we have a string like: "1,2],[3,4],[5,6"
# Replace the sequence "],[" with a space so that each row becomes a token.
rows_str=$(echo "$input" | sed 's/],[/ /g')

# Read the rows into an array.
read -r -a raw_rows <<< "$rows_str"

# Determine number of rows.
rows_count=${#raw_rows[@]}

# Split the first row by comma to determine the number of columns.
IFS=',' read -r -a first_row <<< "${raw_rows[0]}"
cols_count=${#first_row[@]}

# We'll store the matrix in a one-dimensional bash 

# Responses API

https://platform.openai.com/docs/guides/reasoning?api-mode=responses

In [4]:
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = openai.responses.create(
    model=MODEL,
    reasoning={"effort": "medium"}, # reasoning_effort now become reasoning and accept a dictionary
    input=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

rich.print(response.usage)
print(response.output_text)

ResponseUsage(
    input_tokens=103,
    output_tokens=1736,
    output_tokens_details=OutputTokensDetails(reasoning_tokens=1152),
    total_tokens=1839,
    input_tokens_details={'cached_tokens': 0}
)

#!/bin/bash
#
# This script takes a matrix represented as a string in the format:
#   "[1,2],[3,4],[5,6]"
# and prints its transpose in the same format.
#
# Usage:
#   ./transpose.sh '[1,2],[3,4],[5,6]'
#

if [ "$#" -ne 1 ]; then
    echo "Usage: $0 '<matrix>'"
    exit 1
fi

# Get the input matrix string.
matrix_string="$1"

# Remove any whitespace.
matrix_string=$(echo "$matrix_string" | tr -d '[:space:]')

# The input format is a series of rows enclosed in brackets and separated by commas.
# For example: "[1,2],[3,4],[5,6]".
# First, remove the leading "[" and trailing "]" from the entire string.
# Then replace "],[" with a newline so that each row is on its own line.
rows_str=$(echo "$matrix_string" | sed 's/^\[//; s/\]$//; s/\],\[/\n/g')

# Read the rows into an array.
IFS=$'\n' read -d '' -r -a rows <<< "$rows_str"

# Determine the number of rows.
num_rows=${#rows[@]}

# Split the first row by commas to determine the number of columns.
IFS=',' read -r -a first_row_array <<< "${ro